In [ ]:
pip install tensorflow

In [ ]:
import sqlite3
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import MultiLabelBinarizer

# Load data from the SQLite database
def load_data_from_database():
    conn = sqlite3.connect('books.db')
    cursor = conn.cursor()

    # Fetch book data including authors, titles, and genres
    cursor.execute("SELECT authors, title, genres FROM books")
    data = cursor.fetchall()

    conn.close()
    return data

# Preprocess data and create feature representations
def preprocess_data(data):
    # Separate data into authors, titles, and genres
    authors, titles, genres = zip(*data)

    # Use MultiLabelBinarizer to one-hot encode genres
    mlb = MultiLabelBinarizer()
    genre_features = mlb.fit_transform(genres)

    # Create embeddings for authors and titles (example: word embeddings)
    # You can use Word2Vec, FastText, or other embedding techniques
    # For simplicity, here we use random embeddings
    num_authors = len(set(authors))
    num_titles = len(set(titles))

    author_embeddings = np.random.rand(num_authors, 50)  # 50-dimensional embeddings
    title_embeddings = np.random.rand(num_titles, 50)  # 50-dimensional embeddings

    return genre_features, author_embeddings, title_embeddings

# Define a neural network model for recommendation
def create_recommendation_model(num_genres, num_authors, num_titles):
    # Example model architecture
    genre_input = keras.layers.Input(shape=(num_genres,))
    author_input = keras.layers.Input(shape=(50,))
    title_input = keras.layers.Input(shape=(50,))

    # Define layers and connections
    # Customize this architecture based on your requirements
    # Concatenate inputs or use other techniques
    combined_features = keras.layers.Concatenate()([genre_input, author_input, title_input])
    hidden_layer = keras.layers.Dense(128, activation='relu')(combined_features)
    output_layer = keras.layers.Dense(num_titles, activation='softmax')(hidden_layer)

    model = keras.models.Model(inputs=[genre_input, author_input, title_input], outputs=output_layer)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Train the recommendation model
def train_recommendation_model(model, genre_features, author_embeddings, title_embeddings):
    # Load user preferences and book ratings from your dataset
    # User preferences could be a combination of selected books, ratings, etc.

    # Prepare training data (X) and target labels (y)
    X = [genre_features, author_embeddings, title_embeddings]
    y = user_preferences  # User-specific target labels, e.g., book ratings

    # Train the model
    model.fit(X, y, epochs=10, batch_size=32, validation_split=0.2)  # Adjust parameters as needed

# Generate book recommendations using the trained model
def generate_recommendations(model, user_preferences, genre_features, author_embeddings, title_embeddings):
    # Prepare user-specific input
    user_input = [genre_features, author_embeddings, title_embeddings]

    # Predict book recommendations
    recommendations = model.predict(user_input)

    # Sort and select top recommendations
    top_recommendations_indices = recommendations.argsort()[-5:][::-1]  # Adjust the number of recommendations as needed
    top_recommendations = [book_titles[i] for i in top_recommendations_indices]

    return top_recommendations

# Main program
if __name__ == "__main__":
    # Load and preprocess data
    data = load_data_from_database()
    genre_features, author_embeddings, title_embeddings = preprocess_data(data)

    # Create and train the recommendation model
    num_genres = genre_features.shape[1]
    num_authors = author_embeddings.shape[0]
    num_titles = title_embeddings.shape[0]

    model = create_recommendation_model(num_genres, num_authors, num_titles)
    train_recommendation_model(model, genre_features, author_embeddings, title_embeddings)

    # Example: Generate book recommendations based on user preferences
    user_preferences = [user_genre_features, user_author_embedding, user_title_embedding]
    recommendations = generate_recommendations(model, user_preferences, genre_features, author_embeddings, title_embeddings)

    print("Top Book Recommendations:")
    for i, book in enumerate(recommendations):
        print(f"{i+1}. {book}")


In [ ]:
import sqlite3
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import LabelEncoder

# Load data from the SQLite database
def load_data_from_database():
    conn = sqlite3.connect('books.db')
    cursor = conn.cursor()

    # Fetch book data including authors, titles, and genres
    cursor.execute("SELECT authors, title, genres FROM books")
    data = cursor.fetchall()

    conn.close()
    return data

# Preprocess data and create feature representations
def preprocess_data(data):
    # Separate data into authors, titles, and genres
    authors, titles, genres = zip(*data)

    # Use MultiLabelBinarizer to one-hot encode genres
    mlb = MultiLabelBinarizer()
    genre_features = mlb.fit_transform(genres)

    # Use LabelEncoder to encode authors as numerical values
    author_encoder = LabelEncoder()
    author_ids = author_encoder.fit_transform(authors)

    # Create embeddings for titles (example: word embeddings)
    # You can use Word2Vec, FastText, or other embedding techniques
    # For simplicity, here we use random embeddings
    num_titles = len(set(titles))
    title_embeddings = np.random.rand(num_titles, 50)  # 50-dimensional embeddings

    return genre_features, author_ids, title_embeddings

# Define a neural network model for recommendation
def create_recommendation_model(num_genres, num_authors, num_titles):
    # Example model architecture
    genre_input = keras.layers.Input(shape=(num_genres,))
    author_input = keras.layers.Input(shape=(1,))
    title_input = keras.layers.Input(shape=(50,))

    # Define layers and connections
    # Customize this architecture based on your requirements
    combined_features = keras.layers.Concatenate()([genre_input, author_input, title_input])
    hidden_layer = keras.layers.Dense(128, activation='relu')(combined_features)
    output_layer = keras.layers.Dense(num_titles, activation='softmax')(hidden_layer)

    model = keras.models.Model(inputs=[genre_input, author_input, title_input], outputs=output_layer)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Train the recommendation model
def train_recommendation_model(model, genre_features, author_ids, title_embeddings):
    # Prepare training data (X) and target labels (y)
    X = [genre_features, author_ids, title_embeddings]
    y = genre_features  # Use genre_features as target labels (autoencoder)

    # Train the model
    model.fit(X, y, epochs=10, batch_size=32, validation_split=0.2)  # Adjust parameters as needed

# Generate book recommendations using the trained model
def generate_recommendations(model, user_genres, user_authors, genre_features, author_ids, title_embeddings, book_titles):
    # Prepare user-specific input
    user_genre_features = user_genres.reshape(1, -1)
    user_author_id = author_encoder.transform([user_authors])[0]
    user_author_id = np.array([user_author_id])
    user_title_embedding = title_embeddings[0]  # Example user title embedding

    user_input = [user_genre_features, user_author_id, user_title_embedding]

    # Predict book recommendations
    recommendations = model.predict(user_input)

    # Sort and select top recommendations
    top_recommendations_indices = recommendations.argsort()[-5:][::-1]  # Adjust the number of recommendations as needed
    top_recommendations = [book_titles[i] for i in top_recommendations_indices]

    return top_recommendations

# Main program
if __name__ == "__main__":
    # Load and preprocess data
    data = load_data_from_database()
    genre_features, author_ids, title_embeddings = preprocess_data(data)
    book_titles = [title for _, title, _ in data]

    # Prompt the user to select genres and authors of interest
    selected_genres = input("Enter genres of interest (comma-separated): ").split(',')
    selected_authors = input("Enter authors of interest (comma-separated): ").split(',')

    # Filter data based on user selections
    selected_data = [item for item in data if any(genre in item[2] for genre in selected_genres)
                     and any(author in item[0] for author in selected_authors)]

    # Create and train the recommendation model
    num_genres = genre_features.shape[1]
    num_authors = len(set(author_ids))
    num_titles = title_embeddings.shape[0]

    model = create_recommendation_model(num_genres, num_authors, num_titles)
    train_recommendation_model(model, genre_features, author_ids, title_embeddings)

    # Example: Generate book recommendations based on user preferences
    if selected_data:
        user_genres = set(selected_genres)
        user_authors = set(selected_authors)
        recommendations = generate_recommendations(model, user_genres, user_authors, genre_features, author_ids, title_embeddings, book_titles)

        print("Top Book Recommendations:")
        for i, book in enumerate(recommendations):
            print(f"{i+1}. {book}")
    else:
        print("No books match the selected genres and authors.")


In [ ]:
import sqlite3
import random
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

# Connect to the SQLite database
conn = sqlite3.connect('books.db')
c = conn.cursor()

# Get all books from the database
c.execute("SELECT * FROM books")
rows = c.fetchall()

# Assume we have some function to get user ratings.
def get_user_rating(book):
    print(f"Please rate the book '{book[1]}' by {book[2]} (1-5): ")
    return int(input())  # Convert user rating to integer.

# Randomly select 5 books for the user to rate.
random_books = random.sample(rows, 5)
user_ratings = [get_user_rating(book) for book in random_books]

# Prepare data for training the model.
X_train = [[book[2], book[3]] for book in rows]  # author and genre as features.
y_train = [rating for rating in user_ratings]

# Encoding categorical data (author & genre)
label_encoder_author = LabelEncoder()
X_train_encoded_author = label_encoder_author.fit_transform([x[0] for x in X_train])

label_encoder_genre = LabelEncoder()
X_train_encoded_genre= label_encoder_genre.fit_transform([x[1] for x in X_train])

column_transformer = ColumnTransformer(
    [('encoder', OneHotEncoder(), [0, 1])],
    remainder='passthrough'
)

X_train_encoded_full= column_transformer.fit_transform(np.column_stack((X_train_encoded_author,X_train_encoded_genre)))

# Convert sparse matrix to dense
X_train_encoded_full=X_train_encoded_full.toarray()

# Define Keras model
model=Sequential()
model.add(Dense(10,input_dim=X_train_encoded_full.shape[1],activation='relu'))
model.add(Dense(10,activation='relu'))
model.add(Dense(1,activation='linear'))

# Compile model
model.compile(loss='mean_squared_error',optimizer='adam')

# Fit model with our training data.
model.fit(X_train_encoded_full,y_train,epochs=100,batch_size=10)

def recommend_book(user_rating):
    X_test_author_labelencoded=label_encoder_author.transform([user_rating[0]])
    X_test_genre_labelencoded=label_encoder_genre.transform([user_rating[1]])

    X_test_onehotencoded=column_transformer.transform(np.column_stack((X_test_author_labelencoded,X_test_genre_labelencoded)))

    # Convert sparse matrix to dense
    X_test_onehotencoded=X_test_onehotencoded.toarray()

    prediction=model.predict(X_test_onehotencoded)

    recommended_book_index=np.argmax(prediction)

    return rows[recommended_book_index]

print("Recommended Book: ", recommend_book(user_ratings))


In [ ]:
import sqlite3
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer

# Connect to the SQLite database
conn = sqlite3.connect('books.db')

# Read data from database into a pandas DataFrame
df = pd.read_sql_query("SELECT * FROM books", conn)

# Replace missing authors with 'Unknown Author'
df['authors'] = df['authors'].replace({None: 'Unknown Author'})

# Split multiple authors into lists
df['authors'] = df['authors'].str.split(',')

# Convert genre to lowercase and replace 'unknown genre' with 'unknown_genre'
df['genres'] = df['genres'].str.lower().replace({'unknown genre': 'unknown_genre'})

# Initialize MultiLabelBinarizer for author column
mlb_authors = MultiLabelBinarizer()
authors_encoded = mlb_authors.fit_transform(df.pop('authors'))

# Convert author-encoded array into DataFrame and concatenate with original DataFrame
df_authors_encoded = pd.DataFrame(authors_encoded, columns=mlb_authors.classes_)
df_processed = pd.concat([df, df_authors_encoded], axis=1)

print(df_processed.head())



In [ ]:
import sqlite3
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer, LabelEncoder, OneHotEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Connect to the SQLite database
conn = sqlite3.connect('books.db')

# Read data from database into a pandas DataFrame
df = pd.read_sql_query("SELECT * FROM books", conn)

# Replace missing authors with 'Unknown Author'
df['authors'] = df['authors'].replace({None: 'Unknown Author'})

# Split multiple authors into lists
df['authors'] = df['authors'].str.split(',')

# Convert genre to lowercase and replace 'unknown genre' with 'unknown_genre'
df['genres'] = df['genres'].str.lower().replace({'unknown genre': 'unknown_genre'})

# Initialize MultiLabelBinarizer for author column
mlb_authors = MultiLabelBinarizer()
authors_encoded = mlb_authors.fit_transform(df.pop('authors'))

# Convert author-encoded array into DataFrame and concatenate with original DataFrame
df_authors_encoded = pd.DataFrame(authors_encoded, columns=mlb_authors.classes_)
df_processed_author = pd.concat([df, df_authors_encoded], axis=1)

le_genres= LabelEncoder()
genres_encoded= le_genres.fit_transform(df_processed_author.pop('genres'))

one_hot_encoder_genres= OneHotEncoder(sparse=False)
genres_onehotencoded= one_hot_encoder_genres.fit_transform(genres_encoded.reshape(-1,1))

df_genres_onehotencoded=pd.DataFrame(genres_onehotencoded,
                                     columns=[f"genre_{genre}" for genre in le_genres.classes_])

df_processed=pd.concat([df_processed_author.reset_index(drop=True),
                        df_genres_onehotencoded.reset_index(drop=True)],axis=1)


def get_user_rating(book):
    print(f"Please rate the book '{book[1]}' (1-5): ")
    return float(input())  # Convert user rating to float.

random_books_indices=np.random.choice(df_processed.shape[0], size=5)
random_books_ratings=[get_user_rating(df.iloc[i]) for i in random_books_indices]

X_train=df_processed.drop(columns=['title', 'unique_id']).values[random_books_indices].astype(float)
y_train=np.array(random_books_ratings).astype(float)

model=Sequential()
model.add(Dense(10,input_dim=X_train.shape[1],activation='relu'))
model.add(Dense(10,activation='relu'))
model.add(Dense(1))

model.compile(loss='mean_squared_error',optimizer='adam')

model.fit(X_train,y_train,epochs=100,batch_size=10)

# Save the trained model
from tensorflow.keras.models import save_model
save_model(model, 'my_trained_model.h5')

X_test=df_processed.drop(columns=['title', 'unique_id']).values.astype(float)

predictions=model.predict(X_test)
recommended_book_index=np.argmax(predictions)

print("Recommended Book: ", df.iloc[recommended_book_index]['title'])

In [ ]:
from tensorflow.keras.models import load_model

# Load the trained model from a file
model = load_model('my_trained_model.h5')

X_test=df_processed.drop(columns=['title', 'unique_id']).values.astype(float)
predictions=model.predict(X_test)
recommended_book_index=np.argmax(predictions)

print("Recommended Book: ", df.iloc[recommended_book_index]['title'])

In [ ]:
# Connect to the SQLite database
conn = sqlite3.connect('my_database.db')

# Create cursor object
cursor = conn.cursor()

# Create table - RATINGS
cursor.execute('''CREATE TABLE ratings
             ([generated_id] INTEGER PRIMARY KEY,[username] text, [unique_id] integer, [rating] float)''')

conn.commit()


In [ ]:
import os
from tensorflow.keras.models import load_model, save_model
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

def get_user_rating(book):
    print(f"Please rate the book '{book[1]}' (1-5): ")
    return float(input())  # Convert user rating to float.

def train_model(X_train, y_train, model_path):
    model = Sequential()
    model.add(Dense(10,input_dim=X_train.shape[1],activation='relu'))
    model.add(Dense(10,activation='relu'))
    model.add(Dense(1))

    model.compile(loss='mean_squared_error',optimizer='adam')

    # Train the model
    model.fit(X_train,y_train,epochs=100,batch_size=10)

    # Save the trained model
    save_model(model, model_path)

# Connect to the SQLite database
conn = sqlite3.connect('my_database.db')

# Ask user for username
username = input("Enter your username: ")

# Check if user exists in ratings table:
user_exists = pd.read_sql_query(f"SELECT * FROM ratings WHERE username='{username}'", conn).shape[0] > 0

if not user_exists:
   print("New User! Please rate some books.")
else:
   print("Welcome back! Loading your personalized recommendations...")

model_path = f'models/{username}_model.h5'

if os.path.exists(model_path):
     # Load existing recommendation engine from file
     loaded_model = load_model(model_path)
else:
     print("It seems like you haven't rated any books yet.")
     print("Let's start by rating some books!")

random_books_indices=np.random.choice(df_processed.shape[0], size=5)
random_books_ratings=[]
for i in random_books_indices:
      unique_id=df.iloc[i]['unique_id']
      already_rated=pd.read_sql_query(f"SELECT * FROM ratings WHERE username='{username}' AND unique_id='{unique_id}'",conn).shape[0]>0


      if not already_rated:
          random_books_ratings.append((unique_id,get_user_rating(df.iloc[i])))

X_train=df_processed.drop(columns=['title', 'unique_id']).values[random_books_indices].astype(float)
y_train=np.array([rating for _,rating in random_books_ratings]).astype(float)

cursor=conn.cursor()
for unique_id,rating in random_books_ratings:

        cursor.execute(f"INSERT INTO ratings (username, unique_id,rating) VALUES ('{username}', '{unique_id}', {rating})")
        conn.commit()

train_model(X_train,y_train,model_path)

In [ ]:
import os
import sqlite3
import tensorflow as tf
from tensorflow import keras
import numpy as np
from tensorflow.keras.models import load_model, save_model
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer, LabelEncoder, OneHotEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import MultiLabelBinarizer

#read from database containing the books
import sqlite3
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer

# Connect to the SQLite database
conn = sqlite3.connect('books.db')

# Read data from database into a pandas DataFrame
df = pd.read_sql_query("SELECT * FROM books", conn)

# Replace missing authors with 'Unknown Author'
df['authors'] = df['authors'].replace({None: 'Unknown Author'})

# Split multiple authors into lists
df['authors'] = df['authors'].str.split(',')

# Convert genre to lowercase and replace 'unknown genre' with 'unknown_genre'
df['genres'] = df['genres'].str.lower().replace({'unknown genre': 'unknown_genre'})

# Initialize MultiLabelBinarizer for author column
mlb_authors = MultiLabelBinarizer()
authors_encoded = mlb_authors.fit_transform(df.pop('authors'))

# Convert author-encoded array into DataFrame and concatenate with original DataFrame
df_authors_encoded = pd.DataFrame(authors_encoded, columns=mlb_authors.classes_)
df_processed = pd.concat([df, df_authors_encoded], axis=1)

df_processed=pd.concat([df_processed_author.reset_index(drop=True), df_genres_onehotencoded.reset_index(drop=True)],axis=1)

def get_user_rating(book):
    print(f"Please rate the book '{book[1]}' (1-5): ")
    return float(input())  # Convert user rating to float.

def train_model(X_train, y_train, model_path):
    model = Sequential()
    model.add(Dense(10,input_dim=X_train.shape[1],activation='relu'))
    model.add(Dense(10,activation='relu'))
    model.add(Dense(1))

    model.compile(loss='mean_squared_error',optimizer='adam')

    # Train the model
    model.fit(X_train,y_train,epochs=100,batch_size=10)

    # Save the trained model
    save_model(model, model_path)

def recommend_book(model):
   # For simplicity we will just predict rating for every book in our database and return one with highest predicted rating.
   X_all_books = df_processed.drop(columns=['title', 'unique_id']).values.astype(float)
   predictions = loaded_model.predict(X_all_books)

   recommended_book_index = np.argmax(predictions)

   recommended_book_title = df.iloc[recommended_book_index]['title']

   print(f"We recommend you to read: {recommended_book_title}")

# Connect to the SQLite database
conn = sqlite3.connect('my_database.db')

# Ask user for username
username = input("Enter your username: ")

model_path = f'models/{username}_model.h5'

if os.path.exists(model_path):
    # Load existing recommendation engine from file
    loaded_model = load_model(model_path)

    retrain_choice=input("Do you want to retrain your recommendation engine? (yes/no): ")

    if retrain_choice.lower()=='yes':
        print("Let's rate some books!")
        user_ratings=pd.read_sql_query(f"SELECT * FROM ratings WHERE username='{username}'",conn)

        if not user_ratings.empty:
            X_train=df_processed[df_processed['unique_id'].isin(user_ratings['unique_id'])].drop(columns=['title', 'unique_id']).values.astype(float)
            y_train=user_ratings['rating'].values.astype(float)

            train_model(X_train,y_train,model_path)

else:
    # Prompt new users to train a model
    print("Welcome, new user! Let's rate some books and create your personalized recommendation engine.")

    random_books_indices=np.random.choice(df_processed.shape[0], size=5)

    for i in random_books_indices:
        unique_id=df.iloc[i]['unique_id']
        already_rated=pd.read_sql_query(f"SELECT * FROM ratings WHERE username='{username}' AND unique_id='{unique_id}'",conn).shape[0]>0

        if not already_rated:
            rating=get_user_rating(df.iloc[i])
            if rating==0.0:
               continue

            cursor=conn.cursor()
            cursor.execute(f"INSERT INTO ratings (username, unique_id,rating) VALUES ('{username}', '{unique_id}', {rating})")
            conn.commit()

    # After collecting ratings, train the model
    user_ratings=pd.read_sql_query(f"SELECT * FROM ratings WHERE username='{username}'",conn)

    X_train=df_processed[df_processed['unique_id'].isin(user_ratings['unique_id'])].drop(columns=['title', 'unique_id']).values.astype(float)
    y_train=user_ratings['rating'].values.astype(float)

    train_model(X_train,y_train,model_path)

rate_books_choice=input("Do you want to rate some books? (yes/no): ")

if rate_books_choice.lower()=='yes':
      random_books_indices=np.random.choice(df_processed.shape[0], size=5)

      for i in random_books_indices:
          unique_id=df.iloc[i]['unique_id']
          already_rated=pd.read_sql_query(f"SELECT * FROM ratings WHERE username='{username}' AND unique_id='{unique_id}'",conn).shape[0]>0

          if not already_rated:
              rating=get_user_rating(df.iloc[i])
              if rating==0.0:
                 continue

              cursor=conn.cursor()
              cursor.execute(f"INSERT INTO ratings (username, unique_id,rating) VALUES ('{username}', '{unique_id}', {rating})")
              conn.commit()

recommend_book(loaded_model)


In [ ]:
# Ask user for username
username = input("Enter your username: ")

model_path = f'models/{username}_model.h5'

if os.path.exists(model_path):
    # Load existing recommendation engine from file
    loaded_model = load_model(model_path)

    retrain_choice=input("Do you want to retrain your recommendation engine? (yes/no): ")

    if retrain_choice.lower()=='yes':
        print("Let's rate some books!")
        user_ratings=pd.read_sql_query(f"SELECT * FROM ratings WHERE username='{username}'",conn)

        if not user_ratings.empty:
            X_train=df_processed[df_processed['unique_id'].isin(user_ratings['unique_id'])].drop(columns=['title', 'unique_id']).values.astype(float)
            y_train=user_ratings['rating'].values.astype(float)

            train_model(X_train,y_train,model_path)

else:
    # Prompt new users to train a model
    print("Welcome, new user! Let's rate some books and create your personalized recommendation engine.")

    # Code for gathering ratings and training a new model goes here.


In [ ]:
import tkinter as tk
from tkinter import messagebox
from tkinter import simpledialog
import os
import sqlite3
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential, load_model, save_model
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import MultiLabelBinarizer

#read from database containing the books
import sqlite3
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer

# Connect to the SQLite database
conn = sqlite3.connect('books.db')

# Read data from database into a pandas DataFrame
df = pd.read_sql_query("SELECT * FROM books", conn)

# Replace missing authors with 'Unknown Author'
df['authors'] = df['authors'].replace({None: 'Unknown Author'})

# Split multiple authors into lists
df['authors'] = df['authors'].str.split(',')

# Convert genre to lowercase and replace 'unknown genre' with 'unknown_genre'
df['genres'] = df['genres'].str.lower().replace({'unknown genre': 'unknown_genre'})

class BookRecommender:

    def __init__(self):
        self.conn = sqlite3.connect('my_database.db')
        self.username = None
        self.model = None

    def load_model(self):
        model_path = f'models/{self.username}_model.h5'
        if os.path.exists(model_path):
            self.model = load_model(model_path)
        else:
            messagebox.showerror("Error", "No model found for this user.")

    def get_user_rating(self, book):
        rating = simpledialog.askfloat("Input", f"Rate the book '{book[1]}' (1-5):", minvalue=1, maxvalue=5)
        return rating  # Convert user rating to float.

    def rate_books(self):
        random_books_indices = np.random.choice(df_processed.shape[0], size=5)
        for i in random_books_indices:
            book = df.iloc[i]
            rating = self.get_user_rating(book)
            if rating:
                # Save the rating in the database
                cursor = self.conn.cursor()
                cursor.execute(f"INSERT INTO ratings (username, unique_id, rating) VALUES ('{self.username}', '{book[0]}', {rating})")
                self.conn.commit()

    def train_new_model(self):
        print("Welcome, new user! Let's rate some books and create your personalized recommendation engine.")
        self.rate_books()

        user_ratings = pd.read_sql_query(f"SELECT * FROM ratings WHERE username='{self.username}'", self.conn)

        X_train = df_processed[df_processed['unique_id'].isin(user_ratings['unique_id'])].drop(columns=['title', 'unique_id']).values.astype(float)
        y_train = user_ratings['rating'].values.astype(float)

        self.train_model(X_train, y_train)

    def rate_and_train(self):
        self.rate_books()
        user_ratings = pd.read_sql_query(f"SELECT * FROM ratings WHERE username='{self.username}'", self.conn)

        X_train = df_processed[df_processed['unique_id'].isin(user_ratings['unique_id'])].drop(columns=['title', 'unique_id']).values.astype(float)
        y_train = user_ratings['rating'].values.astype(float)

        self.train_model(X_train, y_train)

    def recommend_book(self):
        if self.model is None:
            messagebox.showerror("Error", "No model found for this user.")
            return
   # For simplicity we will just predict rating for every book in our database and return one with highest predicted rating.
        X_all_books = df_processed.drop(columns=['title', 'unique_id']).values.astype(float)
        predictions = self.model.predict(X_all_books)

        recommended_book_index = np.argmax(predictions)

        recommended_book_title = df.iloc[recommended_book_index]['title']

        messagebox.showinfo("Recommendation", f"We recommend you to read: {recommended_book_title}")
class App:

    def __init__(self, root, recommender):
        self.recommender = recommender

        self.username_label = tk.Label(root, text="Enter your username:")
        self.username_label.pack()
        self.username_entry = tk.Entry(root)
        self.username_entry.pack()

        self.load_model_button = tk.Button(root, text="Load Model", command=self.load_model)
        self.load_model_button.pack()

        self.recommend_button = tk.Button(root, text="Get Recommendation", command=self.recommend_book)
        self.recommend_button.pack()

        self.rate_book_button = tk.Button(root, text="Rate a Random Book", command=self.rate_random_book)
        self.rate_book_button.pack()

    def load_model(self):
        username = self.username_entry.get()
        if not username:
            messagebox.showerror("Error", "You must enter a username.")
            return
        self.recommender.username = username
        self.recommender.load_model()

    def recommend_book(self):
        self.recommender.recommend_book()

    def rate_random_book(self):
        random_book_index = np.random.choice(df_processed.shape[0], size=1)[0]
        book = df.iloc[random_book_index]
        rating = self.recommender.get_user_rating(book)
        if rating:
            # Save the rating in the database
            cursor = self.recommender.conn.cursor()
            cursor.execute(f"INSERT INTO ratings (username, unique_id, rating) VALUES ('{self.recommender.username}', '{book['unique_id']}', {rating})")
            self.recommender.conn.commit()

            # Regenerate the model with the new rating
            self.recommender.rate_and_train()

root = tk.Tk()
recommender = BookRecommender()
app = App(root, recommender)
root.mainloop()
